In [23]:
import json
import pandas as pd


"""
TODO:
1. Data sudah bener2 post_rendered, tinggal processing untuk di training BERT
"""

def remove_backslash(text: str) -> str:
    result = text.replace("\\", " ")
    return result

def remove_redundant_white_spaces(text: str) -> str:
    from re import sub
    return sub(r'\s+', ' ', text).strip()


file_path = 'C:/wamp64/www/artofproblemsolving/data/data_post_rendered.csv'
df = pd.read_csv(file_path)

# display_dictionary(data)


In [24]:
"""
NOTE:
Jangan lupa `$...$ hilangin dolar`
Extrak yang diapit dolar (ada yag `$$...$$`) atau lebih dolarnya
"""

# >
biggerthan = "biggerthan"
# <
smallerthan = "smallerthan"
# +
plus_or_positive = "plus_or_positive"
# -
minus_or_negative = "minus_or_negative"
# / == \frac{}{}

# :

# =
equal = "equal"
# !
factorial = "factorial"

# ^
superscript_variable = "superscript_variable"
# _
subscript_variable = "subscript_variable"
# ^_
combined_supersubscript_variable = "combined_supersubscript_variable"

# ABC --> "uppercase_variable uppercase_variable uppercase_variable"
uppercase_variable = "uppercase_variable"
# n, x, y --> lowercase_variable
lowercase_variable = "lowercase_variable"
